# **Compliance GPT with LangChain**

## **Setup**

In [1]:
# import nest_asyncio
# nest_asyncio.apply()

import warnings
warnings.filterwarnings("ignore")

## **Config**

In [2]:
from utils.config import get_config

config = get_config()

In [4]:
USER_ID = 'xmriz2'
# CONVERSATION_ID = 'conv_1'

In [10]:
top_k = 8

## **Define Model**

In [5]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_35_TURBO, embedding_model_name=EmbeddingModelName.EMBEDDING_3_SMALL)

## **Load Vector Store Index**

In [7]:
from database.vector_store.vector_store import ElasticIndexManager

es_bi = ElasticIndexManager(index_name='bi', embed_model=embed_model, config=config)
vector_store_bi = es_bi.load_vector_index()

es_ojk = ElasticIndexManager(index_name='ojk', embed_model=embed_model, config=config)
vector_store_ojk = es_ojk.load_vector_index()

es_ket = ElasticIndexManager(index_name='sikepo-rekam-jejak', embed_model=embed_model, config=config)
vector_store_ket = es_ket.load_vector_index()

es_rek = ElasticIndexManager(index_name='sikepo-ketentuan-terkait', embed_model=embed_model, config=config)
vector_store_rek = es_rek.load_vector_index()

Loaded index 'bi'.
Loaded index 'ojk'.
Loaded index 'sikepo-rekam-jejak'.
Loaded index 'sikepo-ketentuan-terkait'.


In [8]:
from database.vector_store.neo4j_graph_store import Neo4jGraphStore

neo4j_sikepo = Neo4jGraphStore(config=config)
graph = neo4j_sikepo.get_graph()

## **Get Retriever**

In [11]:
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk
from retriever.retriever_bi.retriever_bi import get_retriever_bi
from retriever.retriever_sikepo.lotr_sikepo import lotr_sikepo


retriever_ojk = get_retriever_ojk(vector_store=vector_store_ojk, top_k=top_k,
                                  llm_model=llm_model, embed_model=embed_model, config=config)
retriever_ojk_wo_self = get_retriever_ojk(vector_store=vector_store_ojk, top_k=top_k,
                                          llm_model=llm_model, embed_model=embed_model, config=config, with_self_query=False)
retriever_bi = get_retriever_bi(vector_store=vector_store_bi, top_k=top_k,
                                llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_ket = lotr_sikepo(vector_store=vector_store_ket, top_k=top_k,
                                   llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_ket_wo_self = lotr_sikepo(vector_store=vector_store_ket, top_k=top_k,
                                           llm_model=llm_model, embed_model=embed_model, config=config, with_self_query=False)
retriever_sikepo_rek = lotr_sikepo(vector_store=vector_store_rek, top_k=top_k,
                                   llm_model=llm_model, embed_model=embed_model, config=config)
retriever_sikepo_rek_wo_self = lotr_sikepo(vector_store=vector_store_rek, top_k=top_k,
                                           llm_model=llm_model, embed_model=embed_model, config=config, with_self_query=False)


## **Create Chain**

In [12]:
from database.chat_store import ElasticChatStore

chat_store = ElasticChatStore(k=4, config=config)

In [14]:
from chain.rag_chain import create_combined_answer_chain, create_chain_with_chat_history
from chain.chain_sikepo.graph_cypher_sikepo_chain import graph_rag_chain

graph_chain = graph_rag_chain(llm_model, llm_model, graph=graph)
chain = create_combined_answer_chain(
    llm_model=llm_model,
    graph_chain=graph_chain,
    retriever_ojk=retriever_ojk,
    retriever_bi=retriever_bi,
    retriever_sikepo_ketentuan=retriever_sikepo_ket,
    retriever_sikepo_rekam=retriever_sikepo_rek,
)
chain_wo_self = create_combined_answer_chain(
    llm_model=llm_model,
    graph_chain=graph_chain,
    retriever_ojk=retriever_ojk_wo_self,
    retriever_bi=retriever_bi,
    retriever_sikepo_ketentuan=retriever_sikepo_ket_wo_self,
    retriever_sikepo_rekam=retriever_sikepo_rek_wo_self,
)

# =========== CHAIN HISTORY ===========
chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store,
)
chain_history_wo_self = create_chain_with_chat_history(
    final_chain=chain_wo_self,
    chat_store=chat_store,
)

## **Invoke**

#### **Chat Store Monitor**

In [11]:
# to print chat history
# print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

In [22]:
chat_store.get_conversation_ids_by_user_id(user_id=USER_ID)

[{'id': 'conv_1',
  'title': 'Jika saya ingin membuat peraturan terkait bank dig',
  'date': 1723609343736},
 {'id': 'xmriz2-1723608224505',
  'title': 'Bank Digital',
  'date': 1723608268026},
 {'id': 'xmriz2-1723607806571',
  'title': 'Berapa SWDKLLJ dari Buldozer?',
  'date': 1723607827332}]

In [17]:
# to make the chat history empty
# chat_store.clear_all()

#### **With Question, Answer, and Context**

In [23]:
# from chain.rag_chain import get_response

# response = get_response(
#     chain=chain_history,
#     question="Siapa lionel messi?",
#     user_id=USER_ID,
#     conversation_id="conv_1"
# )

# response

In [ ]:
# print(response['answer'])


In [ ]:
# print(response['context'])

#### **Streaming**

In [24]:
from chain.rag_chain import print_answer_stream2

question = "Jika saya ingin membuat peraturan terkait bank digital peraturan apa yang perlu saya perhatikan?"

await print_answer_stream2(chain=chain_history, question=question,
                     user_id=USER_ID, conversation_id="conv_1")

Peraturan yang perlu diperhatikan jika ingin membuat peraturan terkait bank digital adalah sebagai berikut:

Reference OJK:
Peraturan Otoritas Jasa Keuangan Nomor 12/POJK.03/2018 tentang Penyelenggaraan Layanan Perbankan Digital oleh Bank Umum.
Source: [12/POJK.03/2018](https://www.ojk.go.id/id/regul

CancelledError: 